In [2]:
#inference & save coordinates
!yolo task=detect mode=predict model=/Users/nunofernandes/PycharmProjects/challenge_vc/runs/detect/train8/weights/best.pt conf=0.5 source=/Users/nunofernandes/PycharmProjects/challenge_vc/frames_5 save_txt=True

/Users/nunofernandes/PycharmProjects/emotion_attractiveness/challenge_vc/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Ultralytics YOLOv8.0.196 🚀 Python-3.9.6 torch-2.2.2 CPU (Apple M1)
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

image 1/994 /Users/nunofernandes/PycharmProjects/challenge_vc/frames_5/frame_00000.jpg: 480x800 1 hands, 420.3ms
image 2/994 /Users/nunofernandes/PycharmProjects/challenge_vc/frames_5/frame_00001.jpg: 480x800 1 hands, 316.7ms
image 3/994 /Users/nunofernandes/PycharmProjects/challenge_vc/frames_5/frame_00002.jpg: 480x800 2 handss, 323.0ms
image 4/994 /Users/nunofernandes/PycharmProjects/challenge_vc/frames_5/frame_00003.jpg: 480x800 2 handss, 315.4ms
imag

In [3]:
from main_helper import parse_txt_file


In [9]:
parse_txt_file('/Users/nunofernandes/PycharmProjects/challenge_vc/runs/detect/predict4/labels/frame_00040.txt')

[(0, 0.527595, 0.83699, 0.178359, 0.260086),
 (0, 0.694259, 0.782864, 0.157743, 0.213308)]

In [12]:
import os
def analyze_disappearance(txt_dir):
    """
    Analyzes .txt files in a directory to determine which hand (left or right) disappears more frequently.

    Parameters:
    - txt_dir (str): Directory containing .txt files with YOLO detections.

    Returns:
    - Tuple: (non_dominant_hand, dominant_hand) - IDs of the non-dominant and dominant hands.
    """
    txt_files = sorted([os.path.join(txt_dir, f) for f in os.listdir(txt_dir) if f.endswith('.txt')])

    disappearance_counts = [0, 0]  # Count of disappearances for hand 0 and hand 1

    # Track previous hand detections
    previous_hand_detections = [False, False]

    for txt_file in txt_files:
        detections = parse_txt_file(txt_file)

        # Track hand detections
        hand_detections = [d for d in detections if d[0] == 0 or d[0] == 1]  # Assuming class_id 0 and 1 for hands
        hand_ids_detected = {d[0] for d in hand_detections}

        # Check if any hand disappeared
        for hand_id in range(2):
            if hand_id not in hand_ids_detected and previous_hand_detections[hand_id]:
                # Hand disappeared
                disappearance_counts[hand_id] += 1
                previous_hand_detections[hand_id] = False
            elif hand_id in hand_ids_detected:
                # Hand is detected
                previous_hand_detections[hand_id] = True

    # Determine which hand disappears more frequently
    non_dominant_hand = disappearance_counts.index(max(disappearance_counts))
    dominant_hand = 1 - non_dominant_hand

    print(f"Non-dominant hand: Hand {non_dominant_hand}")
    print(f"Dominant hand: Hand {dominant_hand}")
    return non_dominant_hand, dominant_hand

# Example usage:
txt_dir = '/Users/nunofernandes/PycharmProjects/challenge_vc/runs/detect/predict4/labels'

analyze_disappearance(txt_dir)


Non-dominant hand: Hand 0
Dominant hand: Hand 1


(0, 1)

In [15]:
def detect_phase_1_start(txt_dir, non_dominant_hand, dominant_hand, y_movement_threshold=0.05):
    """
    Detects the start of phase 1 when the non-dominant hand moves in the Y direction in front of the dominant hand.

    Parameters:
    - txt_dir (str): Directory containing .txt files with YOLO detections.
    - non_dominant_hand (int): Class ID of the non-dominant hand.
    - dominant_hand (int): Class ID of the dominant hand.
    - y_movement_threshold (float): Threshold to detect significant Y-axis movement.
    """
    txt_files = sorted([os.path.join(txt_dir, f) for f in os.listdir(txt_dir) if f.endswith('.txt')])

    previous_positions = {0: None, 1: None}
    phase_1_start_frame = None

    for i, txt_file in enumerate(txt_files):
        detections = parse_txt_file(txt_file)

        # Get hand detections
        hand_detections = {d[0]: d[2] for d in detections if d[0] in (non_dominant_hand, dominant_hand)}

        # Get Y centers of detected hands
        if non_dominant_hand in hand_detections and dominant_hand in hand_detections:
            non_dominant_y = hand_detections[non_dominant_hand]
            dominant_y = hand_detections[dominant_hand]

            # Check if non-dominant hand is moving forward (increased Y center)
            if previous_positions[non_dominant_hand] is not None:
                if (non_dominant_y > previous_positions[non_dominant_hand] + y_movement_threshold and
                        non_dominant_y > dominant_y):
                    if phase_1_start_frame is None:
                        phase_1_start_frame = i

            # Update previous positions
            previous_positions[non_dominant_hand] = non_dominant_y

    print(f"Phase 1 start detected at frame: {phase_1_start_frame}")
    return phase_1_start_frame

# Example usage:
txt_dir = '/Users/nunofernandes/PycharmProjects/challenge_vc/runs/detect/predict4/labels'

non_dominant_hand, dominant_hand = analyze_disappearance(txt_dir)
detect_phase_1_start(txt_dir, non_dominant_hand, dominant_hand)

Non-dominant hand: Hand 0
Dominant hand: Hand 1
Phase 1 start detected at frame: None
